In [2]:
import pandas as pd

file_path = "/Users/lzs/Desktop/RU/intern/ultimate_data_filed2.xlsx"  
df = pd.read_excel(file_path)
df_all = df
df.head()
print(df.head)
print(df_all.head)

<bound method NDFrame.head of       zipcode  accommodates  price  security_deposit  cleaning_fee  \
0       10003             4    215             250.0     95.000000   
1       11693             4    140             200.0     81.666667   
2       11693             4    130             500.0     60.000000   
3       11693             6    100             300.0    150.000000   
4       11693             4    105             100.0     81.666667   
...       ...           ...    ...               ...           ...   
3690    10001             5    400             400.0    100.000000   
3691    10001             8    300             200.0    200.000000   
3692    10001             4    250             500.0     75.000000   
3693    10001             4    650            1000.0    100.000000   
3694    10001             4    700            1000.0    100.000000   

      guests_included  extra_people  minimum_nights  availability_30  \
0                   1            25               3      

In [3]:
import pandas as pd

def fill_missing_fees(file_path, output_path):
    # 读取 Excel 文件
    df = pd.read_excel(file_path)
    
    # 去除金额中的 "$" 符号并转换为 float
    df['security_deposit'] = df['security_deposit'].replace('[\$,]', '', regex=True).astype(float)
    df['cleaning_fee'] = df['cleaning_fee'].replace('[\$,]', '', regex=True).astype(float)
    
    # 用相同 zipcode 的平均值填充缺失值
    df['security_deposit'] = df.groupby('zipcode')['security_deposit'].transform(
        lambda x: x.fillna(x.mean())
    )
    df['cleaning_fee'] = df.groupby('zipcode')['cleaning_fee'].transform(
        lambda x: x.fillna(x.mean())
    )
    
    # 保存为新的 Excel 文件
    df.to_excel(output_path, index=False)
    print(f"处理完成，文件已保存至：{output_path}")

# 路径设置
input_path = '/Users/lzs/Desktop/RU/intern/ultimate_data.xlsx'
output_path = '/Users/lzs/Desktop/RU/intern/ultimate_data_filled.xlsx'

# 调用函数
fill_missing_fees(input_path, output_path)


<>:8: SyntaxWarning: invalid escape sequence '\$'
<>:9: SyntaxWarning: invalid escape sequence '\$'
<>:8: SyntaxWarning: invalid escape sequence '\$'
<>:9: SyntaxWarning: invalid escape sequence '\$'
/var/folders/_4/x_cvkmbx1p98627vwgfsgf1h0000gn/T/ipykernel_65366/811985557.py:8: SyntaxWarning: invalid escape sequence '\$'
  df['security_deposit'] = df['security_deposit'].replace('[\$,]', '', regex=True).astype(float)
/var/folders/_4/x_cvkmbx1p98627vwgfsgf1h0000gn/T/ipykernel_65366/811985557.py:9: SyntaxWarning: invalid escape sequence '\$'
  df['cleaning_fee'] = df['cleaning_fee'].replace('[\$,]', '', regex=True).astype(float)


处理完成，文件已保存至：/Users/lzs/Desktop/RU/intern/ultimate_data_filled.xlsx


In [4]:
df_cleaned = df.dropna(axis=1, how='all')

In [ ]:
import pandas as pd



# --- Step 1: 计算四个时间段的 occupancy ---
df['occ_0_30'] = 30 - df['availability_30']
df['occ_0_60'] = 60 - df['availability_60']
df['occ_0_90'] = 90 - df['availability_90']
df['occ_0_365'] = 365 - df['availability_365']

df['occ_31_60'] = df['occ_0_60'] - df['occ_0_30']
df['occ_61_90'] = df['occ_0_90'] - df['occ_0_60']
df['occ_91_365'] = df['occ_0_365'] - df['occ_0_90']

# --- Step 2: 评分字段列表（rating 是 100 满分，需转为 10） ---
df['review_scores_rating'] = df['review_scores_rating'] / 10

review_cols = [
    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
    'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value'
]

# --- Step 3: 计算归一化评分 和 退订调整系数 ---
df['review_score_normalized'] = df[review_cols].mean(axis=1) / 10
df['adjust_factor'] = 1 - 0.25 * df['review_score_normalized']  

# --- Step 4: 定义基础退订率（四个时间段） ---
base_cxl_rates = {
    'occ_0_30': 0.20,
    'occ_31_60': 0.40,
    'occ_61_90': 0.60,
    'occ_91_365': 0.80,
}

# --- Step 5: 计算每段有效 occupancy ---
for col, rate in base_cxl_rates.items():
    df[f'{col}_effective'] = df[col] * (1 - rate * df['adjust_factor'])

# --- Step 6: 汇总总入住天数并四舍五入 ---
df['total_effective_occupancy'] = df[[f'{col}_effective' for col in base_cxl_rates]].sum(axis=1).round(0).astype(int)


cols_to_round = [
    'occ_0_30', 'occ_31_60', 'occ_61_90', 'occ_91_365',
    'review_score_normalized', 'adjust_factor'
]
df[cols_to_round] = df[cols_to_round].round(0).astype(int)


In [6]:



print(df.head())

   zipcode  accommodates  price  security_deposit  cleaning_fee  \
0    10003             4    215             250.0     95.000000   
1    11693             4    140             200.0     81.666667   
2    11693             4    130             500.0     60.000000   
3    11693             6    100             300.0    150.000000   
4    11693             4    105             100.0     81.666667   

   guests_included  extra_people  minimum_nights  availability_30  \
0                1            25               3                3   
1                4            20               2                0   
2                4            45               1               21   
3                6             0               1                4   
4                2            40               1               21   

   availability_60  ...  occ_31_60  occ_61_90 occ_91_365  \
0                6  ...         27         13        125   
1                0  ...         30         30        159   
2 

In [7]:


def estimate_annual_revenue(price, accommodates, guests_included, 
                            extra_people, cleaning_fee, security_deposit, 
                            total_effective_occupancy):
    # 1. 缺失值处理 + 类型转换
    price = float(price) if pd.notna(price) else 0
    accommodates = int(accommodates) if pd.notna(accommodates) else 1
    guests_included = int(guests_included) if pd.notna(guests_included) else 0
    extra_people = float(extra_people) if pd.notna(extra_people) else 0
    cleaning_fee = float(cleaning_fee) if pd.notna(cleaning_fee) else 0
    security_deposit = float(security_deposit) if pd.notna(security_deposit) else 0
    occupied_days = int(total_effective_occupancy) if pd.notna(total_effective_occupancy) else 0

    # 2. 各部分计算
    base_revenue = occupied_days * price

    extra_guest_count = max(0, accommodates - guests_included)
    extra_guest_fee = extra_guest_count * extra_people * occupied_days

    total_cleaning_fee = cleaning_fee * occupied_days / 7 if occupied_days > 0 else 0

    total_security_deposit = (security_deposit * 0.01) * occupied_days

    # 3. 总收入
    estimated_revenue = base_revenue + extra_guest_fee + total_cleaning_fee + total_security_deposit
    return estimated_revenue


df["estimated_annual_revenue"] = df.apply(
    lambda row: estimate_annual_revenue(
        row["price"], 
        row["accommodates"], 
        row["guests_included"], 
        row["extra_people"], 
        row["cleaning_fee"], 
        row["security_deposit"], 
        row["total_effective_occupancy"]  
    ), axis=1
)


print(df['estimated_annual_revenue'])


0        40095.357143
1        25816.000000
2         6317.142857
3        10700.857143
4        44277.333333
            ...      
3690      9972.285714
3691     70411.714286
3692     60848.571429
3693     16182.857143
3694    175277.142857
Name: estimated_annual_revenue, Length: 3695, dtype: float64


In [8]:
df_all = df.copy()
output_df = df_all[["zipcode", "last_review", "estimated_annual_revenue"]]

output_path = "/Users/lzs/Desktop/RU/intern/final_1.xlsx"  
output_df.to_excel(output_path, index=False)
print(f"已将 estimated_occupancy_365 和 estimated_annual_revenue 计算完成，并保存至 {output_path}")


已将 estimated_occupancy_365 和 estimated_annual_revenue 计算完成，并保存至 /Users/lzs/Desktop/RU/intern/final_1.xlsx


In [9]:

file_path = "/Users/lzs/Desktop/RU/intern/output_1.xlsx"  #
df_all = pd.read_excel(file_path)

zipcode_summary = df_all.groupby("zipcode")["estimated_annual_revenue"].agg(["mean", "count"]).reset_index()

zipcode_summary.columns = ["zipcode", "average_revenue", "count"]

output_path = "/Users/lzs/Desktop/RU/intern/final_2.xlsx"
zipcode_summary.to_excel(output_path, index=False)

print(f"已计算每个 zipcode 的总收入和平均收入，并保存至 {output_path}")


已计算每个 zipcode 的总收入和平均收入，并保存至 /Users/lzs/Desktop/RU/intern/final_2.xlsx


In [10]:
import pandas as pd

file_path = "/Users/lzs/Desktop/RU/intern/final_1.xlsx"
df = pd.read_excel(file_path)

df["last_review"] = pd.to_datetime(df["last_review"], errors="coerce")


df["last_review"] = df["last_review"].dt.strftime("%Y/%-m/%-d") 


output_path = "/Users/lzs/Desktop/RU/intern/final_1_converted.xlsx"
df.to_excel(output_path, index=False)

/var/folders/_4/x_cvkmbx1p98627vwgfsgf1h0000gn/T/ipykernel_65366/2887301808.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["last_review"] = pd.to_datetime(df["last_review"], errors="coerce")


In [11]:
import pandas as pd


file_1 = "/Users/lzs/Desktop/RU/intern/final_1_converted.xlsx"  
file_2 = "/Users/lzs/Desktop/RU/intern/Zip_Zhvi_2bedroom_converted.xlsx"  


df_1 = pd.read_excel(file_1, dtype={"zipcode": str, "last_review": str})


df_2 = pd.read_excel(file_2, dtype={"zipcode": str})


date_columns = df_2.columns[1:]


merged_df = df_1.merge(df_2, on="zipcode", how="left")


def find_closest_value(row, date_cols):
    if pd.isna(row["last_review"]):
        return 0  
    
   
    last_review = row["last_review"]
    
    
    closest_date_col = min(date_cols, key=lambda col: abs(pd.to_datetime(col, format="%b-%y", errors="coerce") - pd.to_datetime(last_review, errors="coerce")).days)
    
    return row.get(closest_date_col, 0)  


merged_df["matched_value"] = merged_df.apply(lambda row: find_closest_value(row, date_columns), axis=1)


output_df = merged_df[["zipcode", "last_review", "matched_value"]]


output_path = "/Users/lzs/Desktop/RU/intern/matched_zipcode_values_1.xlsx"
output_df.to_excel(output_path, index=False)



print(f"匹配完成，文件已保存至: {output_path}")


KeyboardInterrupt: 

In [12]:
import pandas as pd

# 读取两个 Excel
df1 = pd.read_excel('/Users/lzs/Desktop/RU/intern/final_1_converted.xlsx')  # 第一个图的表
df2 = pd.read_excel('/Users/lzs/Desktop/RU/intern/matched_zipcode_values_1.xlsx')  # 第二个图的表

# merge 关联，按zipcode和last_review匹配
df_merged = df1.merge(df2, on=['zipcode', 'last_review'], how='left')

# 保存结果
df_merged.to_excel('/Users/lzs/Desktop/RU/intern/matched_zipcode_values_output.xlsx', index=False)


In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# borough-color 映射
borough_color = {
    'Manhattan': '#b8860b',
    'Brooklyn': '#1f77b4',
    'Queens': '#2ca02c',
    'Staten Island': '#8b4513'
}

# zipcode 到 borough 映射（可根据数据补全）
zipcode_to_borough = {
    '10036': 'Manhattan',
    '11231': 'Brooklyn',
    '11215': 'Brooklyn',
    '11217': 'Brooklyn',
    '10003': 'Manhattan',
    '10025': 'Manhattan',
    '10011': 'Manhattan',
    '11201': 'Brooklyn',
    '10023': 'Manhattan',
    '10014': 'Manhattan'
}

# 添加 borough 和颜色信息
top_zipcodes['zipcode'] = top_zipcodes['zipcode'].astype(str)
top_zipcodes['borough'] = top_zipcodes['zipcode'].map(zipcode_to_borough)
top_zipcodes['color'] = top_zipcodes['borough'].map(borough_color)

# 画图
plt.figure(figsize=(10, 6))
plt.bar(top_zipcodes['zipcode'], top_zipcodes['payback'], color=top_zipcodes['color'])
plt.xlabel('Zipcode')
plt.ylabel('Average Payback of Top 10')
plt.title('Top 10 Zipcodes with Highest Avg Payback (Top 10 within Each)')
plt.xticks(rotation=45)

# 添加图例
legend_patches = [mpatches.Patch(color=color, label=borough) for borough, color in borough_color.items()]
plt.legend(handles=legend_patches, title="Borough", loc='upper right')

plt.tight_layout()
plt.show()


NameError: name 'top_zipcodes' is not defined